# Свёрточные нейросети

In [ ]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

### 1) Подготока датасета

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomAffine(degrees=10, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=5),
    transforms.ToTensor(),
    transforms.Normalize((0.1,), (0.1,))
])

train_dataset = datasets.MNIST(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root="./data", train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, num_workers=4)

### 2) Создание нейронной сети

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.pool(self.relu1(self.conv1(x)))
        x = self.pool(self.relu2(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = self.dropout(self.relu3(self.fc1(x)))
        x = self.fc2(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 3) Тестирование модели

In [ ]:
def evaluate(model, loader, device):
  model.eval()
  correct = 0
  total = 0
  with torch.no_grad():
    for images, labels in loader:
      images, labels = images.to(device), labels.to(device)
      outputs = model(images)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()
  return correct / total

### 4) Ранняя остановка

In [ ]:
class EarlyStopping:
  def __init__(self, patience=5, min_delta=0):
    self.patience = patience
    self.min_delta = min_delta
    self.best_loss = None
    self.counter = 0

  def should_stop(self, val_loss):
    if self.best_loss is None or val_loss < self.best_loss - self.min_delta:
      self.best_loss = val_loss
      self.counter = 0
    else:
      self.counter += 1
      if self.counter >= self.patience:
        return True
    return False

### 5) ModelCheckpoint

In [ ]:
class ModelCheckpoint:
  def __init__(self, filepath, monitor="val_loss", mode="min", verbose=True):
    self.filepath = filepath
    self.monitor = monitor
    self.mode = mode
    self.best_score = None
    self.verbose = verbose

  def save_checkpoint(self, model, epoch, val_score):
        if self.best_score is None or (
            self.mode == 'min' and val_score < self.best_score or
            self.mode == 'max' and val_score > self.best_score
        ):
            self.best_score = val_score
            torch.save(model.state_dict(), self.filepath)
            if self.verbose:
                print(f"✅ Модель сохранена на эпохе {epoch+1} с {self.monitor}: {val_score:.4f}")


### 6) Тренировка и тестирование модели

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
early_stopping = EarlyStopping(patience=5)
checkpoint = ModelCheckpoint(filepath='best_model.pth', monitor='val_loss', mode='min', verbose=True)

for epoch in range(20):
  model.train()
  running_loss = 0.0
  for images, labels in train_loader:
    images, labels = images.to(device), labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
  avg_train_loss = running_loss / len(train_loader)
  val_accuracy = evaluate(model, test_loader, device)

  print(f"Эпоха [{epoch+1}/{50}], Потери: {avg_train_loss:.4f}, Точность на валидации: {val_accuracy:.4f}")

  checkpoint.save_checkpoint(model, epoch, avg_train_loss)

  if early_stopping.should_stop(avg_train_loss):
    print("Ранняя остановка сработала.")
    break

model.load_state_dict(torch.load('best_model.pth'))

test_accuracy = evaluate(model, test_loader, device)
print(f"Точность на тесте: {test_accuracy:.4f}")

Эпоха [1/50], Потери: 0.6568, Точность на валидации: 0.9311
✅ Модель сохранена на эпохе 1 с val_loss: 0.6568
Эпоха [2/50], Потери: 0.3177, Точность на валидации: 0.9518
✅ Модель сохранена на эпохе 2 с val_loss: 0.3177
Эпоха [3/50], Потери: 0.2573, Точность на валидации: 0.9597
✅ Модель сохранена на эпохе 3 с val_loss: 0.2573
Эпоха [4/50], Потери: 0.2274, Точность на валидации: 0.9672
✅ Модель сохранена на эпохе 4 с val_loss: 0.2274
Эпоха [5/50], Потери: 0.2086, Точность на валидации: 0.9676
✅ Модель сохранена на эпохе 5 с val_loss: 0.2086
Эпоха [6/50], Потери: 0.1893, Точность на валидации: 0.9685
✅ Модель сохранена на эпохе 6 с val_loss: 0.1893
Эпоха [7/50], Потери: 0.1819, Точность на валидации: 0.9702
✅ Модель сохранена на эпохе 7 с val_loss: 0.1819
Эпоха [8/50], Потери: 0.1708, Точность на валидации: 0.9721
✅ Модель сохранена на эпохе 8 с val_loss: 0.1708
Эпоха [9/50], Потери: 0.1658, Точность на валидации: 0.9710
✅ Модель сохранена на эпохе 9 с val_loss: 0.1658
Эпоха [10/50], Поте